In [26]:
!pip install neo4j

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:

import json
from neo4j import GraphDatabase
import pandas as pd


import urllib
import requests
import os
import sys

global library_dict
requests.packages.urllib3.disable_warnings()


In [2]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Amish_2020_Tesla"))


In [5]:
## Create Indexes

In [40]:
with driver.session(database="nationwide") as session:
    tx = session.begin_transaction()
    tx.run(
            "CREATE INDEX rel_first_seen IF NOT EXISTS FOR ()-[r:IN_HOUSEHOLD]-() ON (r.firstSeen)").consume()
    tx.run(
            "CREATE INDEX rel_last_seen IF NOT EXISTS FOR ()-[r:IN_HOUSEHOLD]-() ON (r.lastSeen)").consume()
    tx.run(
            "CREATE INDEX rel_firstandlast_seen IF NOT EXISTS FOR ()-[r:IN_HOUSEHOLD]-() ON (r.firstSeen, r.lastSeen)").consume()
    tx.run(
            "CREATE INDEX customer_id_idx IF NOT EXISTS FOR (c:Customer) ON (c.customer_id)").consume()
    tx.run(
            "CREATE INDEX policy_id_idx IF NOT EXISTS FOR (p:Policy) ON (p.policy_id)").consume()
    tx.run(
            "CREATE INDEX policy_diaid_idx IF NOT EXISTS FOR (p:Policy) ON (p.diaid)").consume()
    tx.run(
            "CREATE INDEX rel_duration IF NOT EXISTS FOR ()-[r:IN_HOUSEHOLD]-() ON (r.lastSeenDuration)").consume
    tx.commit()

In [6]:
def loadCustomerData():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "load csv with headers from 'file:/customer.csv' as row merge (c1:Customer {customer_id:row.customer_id}) ON CREATE SET c1.household_id = toInteger(row.household_id);",
            ).consume()
        tx.commit()



In [7]:
def loadPolicyData():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "load csv with headers from 'file:/policy.csv' as row merge (p:Policy {diaid:row.diaid}) ON CREATE SET p.st_code = row.st_code, p.policy_id = row.pol, p.household_id = toInteger(row.household_id), p.type = row.type;",
            ).consume()
        tx.commit()



In [8]:
def loadPolicyCustomerData():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "load csv with headers from 'file:/policy_customer.csv' as row MATCH (c:Customer {customer_id: row.customer_id}) MATCH (p:Policy {diaid:row.diaid}) CALL apoc.merge.relationship(c, row.reltype,{},{role_start:row.role_start, role_end:row.role_end},p,{}) yield rel return null;",
            ).consume()
        tx.commit()



In [9]:
## Clean entire Graph

In [10]:
def cleanEntireGraph():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "match (n) detach delete n;" ).consume()
        tx.commit()

In [9]:
## Clean Graph but leave Bloom Perspective

In [10]:
def cleanEntireGraphLeaveBloom():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "match (n:Policy) detach delete n;" ).consume()
        tx.run(
            "match (n:Household) detach delete n;" ).consume()
        tx.run(
            "match (n:Customer) detach delete n;" ).consume()
        tx.run(
            "match (n:Inactive_Policy) detach delete n;" ).consume()
        tx.commit()

In [11]:
###loadTestData()

loadCustomerData()


In [12]:
loadPolicyData()

In [13]:
loadPolicyCustomerData()

In [14]:
### Fix Labels

In [15]:
def fixLabels():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "match (p:Policy) where p.type contains 'AA' set p:AA",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'BT' set p:BT",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'CA' set p:CA",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'CC' set p:CC",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'CO' set p:CO",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'DP' set p:DP",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'HO' set p:HO",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'LF' set p:LF",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'MC' set p:MC",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'MH' set p:MH",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'MT' set p:MT",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'RT' set p:RT",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'SL' set p:SL",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'SM' set p:SM",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'TT' set p:TT",
            ).consume()
        tx.run(
            "match (p:Policy) where p.type contains 'UB' set p:UB",
            ).consume()
        tx.commit()

In [16]:
fixLabels();

In [7]:
def createInMemoryGraph():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "CALL gds.graph.create('policy_customer',['Customer','Policy'],['PNI', 'SNI','OWNER','SPOUSE'], {nodeProperties: 'household_id'}) YIELD graphName, nodeProjection, nodeCount AS nodes, relationshipCount AS rels RETURN graphName, nodes, rels;").consume()
        tx.commit()

In [8]:
def runWCC():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run(
            "CALL gds.wcc.write('policy_customer', { writeProperty: 'household_id', seedProperty:'household_id' }) YIELD nodePropertiesWritten, componentCount; ").consume()
        tx.commit()

In [74]:
def updateHouseholds():
    with driver.session(database="nationwide") as session: 
        tx = session.begin_transaction()
        tx.run(
            "call apoc.periodic.iterate('MATCH (c:Customer) return id(c) as idc','match (c) where id(c) = idc with c MERGE (hh:Household {household_id:c.household_id}) ON CREATE SET hh.create_date = date() MERGE (c)-[r:IN_HOUSEHOLD]->(hh) ON CREATE SET  r.firstSeen=datetime(), r.lastSeen=datetime() ON MATCH SET r.lastSeen=datetime()',{batchSize:1000,parallel:false});" ).consume()
        tx.commit()

In [78]:
def updateHouseholdsHistory():
    with driver.session(database="nationwide") as session: 
        tx = session.begin_transaction()
        tx.run(
            "call apoc.periodic.iterate('MATCH (c:Customer) return id(c) as idc','match (c)-[r:IN_HOUSEHOLD]->(hh:Household) with max(r.lastSeen) as lastDate, c match (c:Customer)-[r:IN_HOUSEHOLD]->(hh:Household) with r where duration.inSeconds(r.lastSeen, lastDate).seconds > 0 with type(r) as oldType, \"PREVIOUS_\" + type(r) as newType, collect(r) as allR CALL apoc.refactor.rename.type(oldType, newType ,allR) yield committedOperations return committedOperations;',{batchSize:1000,parallel:false});" ).consume()
        tx.commit()

In [10]:
def calculateSeenDurations():
    with driver.session(database="nationwide") as session: 
        tx = session.begin_transaction()
        tx.run(
            "call apoc.periodic.iterate('MATCH (h:Household) return id(h) as idh','match (h) where id(h) = idh with h MATCH (h)<-[r:IN_HOUSEHOLD]-(c) SET r.lastSeenDuration = duration.inDays(r.firstSeen,r.lastSeen).days;',{batchSize:1000,parallel:false});" ).consume()
        tx.commit()

In [11]:
## Drop In-Memory Graph

In [12]:
def dropInMemoryGraph():
    with driver.session(database="nationwide") as session:
        tx = session.begin_transaction()
        tx.run("call gds.graph.drop('policy_customer');").consume()
        tx.commit()

In [45]:
createInMemoryGraph();

In [ ]:
## Run WCC

In [46]:
runWCC();

In [47]:
## Update Household Ids
## Update the durations on the IN_HOUSEHOLD relationship

In [48]:
updateHouseholds()
calculateSeenDurations()

In [101]:
## Drop In-Memory Graph

In [49]:
dropInMemoryGraph();

In [49]:
## Add a new customer to existing policy.
## Use case 2
## Customer -> neo4j910001516945
## Policy -> diaid 196510063

In [141]:
driver.close()